## 1.导入相关工具、读取数据集

In [74]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
plt.rcParams['font.sans-serif'] = ['Microsoft YaHei']  #显示中文
plt.rcParams['axes.unicode_minus'] = False  #允许显示负数
import seaborn as sns

In [75]:
data = pd.read_csv('/home/mw/input/taobao6982/tianchi_mobile_recommend_train_user.csv')

In [76]:
data.head()

,user_id,item_id,behavior_type,user_geohash,item_category,time
0,98047837,232431562,1,NaN,4245,2014-12-06 02
1,97726136,383583590,1,NaN,5894,2014-12-09 20
2,98607707,64749712,1,NaN,2883,2014-12-18 11
3,98662432,320593836,1,96nn52n,6562,2014-12-06 10
4,98145908,290208520,1,NaN,13926,2014-12-16 21


* 字段含义：分别表示：用户身份，商品的ID，用户行为类型（包括点击、收藏、加购物车、支付四种行为，分别用数字1，2，3，4来代表），地理位置，品类ID，用户行为发生的时间。

In [77]:
data.info()  # 查看基本信息

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12256906 entries, 0 to 12256905
Data columns (total 6 columns):
user_id          int64
item_id          int64
behavior_type    int64
user_geohash     object
item_category    int64
time             object
dtypes: int64(4), object(2)
memory usage: 561.1+ MB


## 2.数据预处理，包括删除重复值，处理缺省值和数据类型转换

In [78]:
# 删除重复值
data.drop_duplicates(inplace = True)

In [79]:
# 查看缺失值
data.isnull().sum()

user_id                0
item_id                0
behavior_type          0
user_geohash     4308015
item_category          0
time                   0
dtype: int64

In [80]:
data.drop('user_geohash',axis=1,inplace = True)
# 地理位置信息无关，直接删除整个一行

In [81]:
# 查看删除之后的结果
data.head()

,user_id,item_id,behavior_type,item_category,time
0,98047837,232431562,1,4245,2014-12-06 02
1,97726136,383583590,1,5894,2014-12-09 20
2,98607707,64749712,1,2883,2014-12-18 11
3,98662432,320593836,1,6562,2014-12-06 10
4,98145908,290208520,1,13926,2014-12-16 21


In [82]:
# 重置索引
data = data.reset_index(drop = True)  # 每次删除整个一行后都需要重置索引

In [83]:
# 时间变量预处理
data['time'] = pd.to_datetime(data['time']) # 类型转换 从object到时间类型
data['date'] = data['time'].dt.date
data['date'] = pd.to_datetime(data['date'])
data['hour'] = data['time'].dt.hour
data.head()

,user_id,item_id,behavior_type,item_category,time,date,hour
0,98047837,232431562,1,4245,2014-12-06 02:00:00,2014-12-06,2
1,97726136,383583590,1,5894,2014-12-09 20:00:00,2014-12-09,20
2,98607707,64749712,1,2883,2014-12-18 11:00:00,2014-12-18,11
3,98662432,320593836,1,6562,2014-12-06 10:00:00,2014-12-06,10
4,98145908,290208520,1,13926,2014-12-16 21:00:00,2014-12-16,21


In [84]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8164040 entries, 0 to 8164039
Data columns (total 7 columns):
user_id          int64
item_id          int64
behavior_type    int64
item_category    int64
time             datetime64[ns]
date             datetime64[ns]
hour             int64
dtypes: datetime64[ns](2), int64(5)
memory usage: 436.0 MB


In [85]:
data.item_id = data.item_id.astype(str)
data['item_category'] = data['item_category'].astype(str)   # 类型转换
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8164040 entries, 0 to 8164039
Data columns (total 7 columns):
user_id          int64
item_id          object
behavior_type    int64
item_category    object
time             datetime64[ns]
date             datetime64[ns]
hour             int64
dtypes: datetime64[ns](2), int64(3), object(2)
memory usage: 436.0+ MB


In [86]:
# 删除之前已经处理过的时间这一列
data.drop(labels = 'time',axis = 1, inplace = True)

In [87]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8164040 entries, 0 to 8164039
Data columns (total 6 columns):
user_id          int64
item_id          object
behavior_type    int64
item_category    object
date             datetime64[ns]
hour             int64
dtypes: datetime64[ns](1), int64(3), object(2)
memory usage: 373.7+ MB


In [88]:
data.head()

,user_id,item_id,behavior_type,item_category,date,hour
0,98047837,232431562,1,4245,2014-12-06,2
1,97726136,383583590,1,5894,2014-12-09,20
2,98607707,64749712,1,2883,2014-12-18,11
3,98662432,320593836,1,6562,2014-12-06,10
4,98145908,290208520,1,13926,2014-12-16,21


## 3.数据分析

### 3.1 用户行为分析

### 3.1.1 时间维度活跃度分析

In [89]:
daily_visits_s = data.groupby(by = 'date')['user_id'].count() # 日访问量
daily_visits_s.head()

date
2014-11-18    235493
2014-11-19    233144
2014-11-20    226523
2014-11-21    213894
2014-11-22    232994
Name: user_id, dtype: int64

In [90]:
# 日独立访客量
unique_visitors_daily_s = data.groupby(by = 'date')['user_id'].nunique()  # 去重处理
unique_visitors_daily_s.head()

date
2014-11-18    6343
2014-11-19    6420
2014-11-20    6333
2014-11-21    6276
2014-11-22    6187
Name: user_id, dtype: int64

In [91]:
# 人均访问量
vistis_per_capita_s = daily_visits_s / unique_visitors_daily_s
vistis_per_capita_s.head()

date
2014-11-18    37.126439
2014-11-19    36.315265
2014-11-20    35.768672
2014-11-21    34.081262
2014-11-22    37.658639
Name: user_id, dtype: float64

In [92]:
# 构建对应的dataframe
df = pd.concat((daily_visits_s,unique_visitors_daily_s,vistis_per_capita_s),axis = 1)
df.columns = ['daily_visits_s','unique_visitors_daily_s','vistis_per_capita_s']
df.head()

,daily_visits_s,unique_visitors_daily_s,vistis_per_capita_s
date,,,
2014-11-18,235493,6343,37.126439
2014-11-19,233144,6420,36.315265
2014-11-20,226523,6333,35.768672
2014-11-21,213894,6276,34.081262
2014-11-22,232994,6187,37.658639


In [93]:
# 绘制日访问量的折线图
plt.plot(df.index,df.daily_visits_s)
plt.title('daily_visits_s')
plt.xticks(rotation = 30) # 横坐标日期标记太长，如果不旋转30度，则会重叠
plt.show()

<Figure size 432x288 with 1 Axes>

In [94]:
# 绘制日独立访客量的折线图
plt.plot(df.index,df.unique_visitors_daily_s)
plt.title('unique_visitors_daily_s')
plt.xticks(rotation = 30) # 横坐标日期标记太长，如果不旋转30度，则会重叠
plt.show()

<Figure size 432x288 with 1 Axes>

In [95]:
# 绘制人均访问量的折线图
plt.plot(df.index,df.vistis_per_capita_s)
plt.title('vistis_per_capita_s')
plt.xticks(rotation = 30) # 横坐标日期标记太长，如果不旋转30度，则会重叠
plt.show()

<Figure size 432x288 with 1 Axes>

* 分析：在双十二当天用户活跃度达到顶峰，但在12月5号的时候用户活跃度就开始上升了。

双十二当天用户活跃度分析

In [96]:
# 取出双十二当天的数据
data_1212 = data.loc[data['date'] == '2014-12-12']

In [97]:
# 双十二当天每小时访问量
visits_hour_s = data_1212.groupby(by='hour')['user_id'].count()
# 双十二当天每小时访客量
visitors_hour_s = data_1212.groupby(by='hour')['user_id'].nunique()
# 双十二当天每小时人均访问量
v_capita_hour_s = visits_hour_s/visitors_hour_s

In [98]:
# 构建对应的dataframe
df1 = pd.concat((visits_hour_s,visitors_hour_s,v_capita_hour_s),axis = 1)
df1.columns = ['visits_hour_s','visitors_hour_s','v_capita_hour_s']
df1.head()

,visits_hour_s,visitors_hour_s,v_capita_hour_s
hour,,,
0,25075,1569,15.981517
1,11388,811,14.041924
2,5956,411,14.491484
3,3177,255,12.458824
4,2567,211,12.165877


In [99]:
# 绘制每小时访问量的折线图
plt.plot(df1.index,df1.visits_hour_s)
plt.title('visits_hour_s')
plt.show()

<Figure size 432x288 with 1 Axes>

In [100]:
# 绘制每小时访客量的折线图
plt.plot(df1.index,df1.visitors_hour_s)
plt.title('visitors_hour_s')
plt.show()

<Figure size 432x288 with 1 Axes>

In [101]:
# 绘制每小时人均访问量的折线图
plt.plot(df1.index,df1.v_capita_hour_s)
plt.title('v_capita_hour_s')
plt.show()

<Figure size 432x288 with 1 Axes>

* 可以看出，用户当天活跃度在18点以后不断上升并达到峰值，可以在18点以后加大推送和促销力度

用户行为活跃度变化  
对用户的四种行为进行分析（1，2，3，4分别表示点击、收藏、加购物车和支付四种行为）

In [102]:

# 建立透视表
pv_df = data.pivot_table(index = 'date',columns = 'behavior_type',aggfunc = 'size',fill_value = 0)
pv_df.head()

behavior_type,1,2,3,4
date,,,,
2014-11-18,215480,6797,9800,3416
2014-11-19,213108,7079,9607,3350
2014-11-20,206740,7063,9552,3168
2014-11-21,196121,6722,8328,2723
2014-11-22,213292,7001,9493,3208


In [103]:
# 创建一个新的figure对象
plt.figure(figsize=(18, 12))

# 画出每种行为的折线图
plt.plot(pv_df.index, pv_df[1], color='blue', label='点击')
plt.plot(pv_df.index, pv_df[2], color='green', label='收藏')
plt.plot(pv_df.index, pv_df[3], color='red', label='加购物车')
plt.plot(pv_df.index, pv_df[4], color='purple', label='支付')

# 添加图例
plt.legend()
plt.xticks(rotation = 30)
# 显示图形
plt.show()

<Figure size 1296x864 with 1 Axes>

* 结论：四种行为都在双十二当天达到顶峰，这是意料之中的事情。可以看出支付量大于收藏量，可能是用户在推送的影响下购买了其他并未收藏的商品。  
* 建议：可以加大推送和促销的力度，或者增加拼单凑单的机会。

### 3.1.2 留存率分析

In [104]:
s = data['behavior_type'].value_counts()
s

1    7479078
3     333371
2     240919
4     110672
Name: behavior_type, dtype: int64

In [105]:
# 封装到具体的df中
s.index = ['点击','加购物车','收藏','支付']
df2 = pd.DataFrame([s.index,s]).T
df2.columns = ['用户行为','访问量'] # 列
df2

,用户行为,访问量
0,点击,7479078
1,加购物车,333371
2,收藏,240919
3,支付,110672


In [106]:
#计算从点击到收藏，从收藏到加购，从加购到支付的转换率
temp1 = df2['访问量'][1:].values # array([333371, 240919, 110672], dtype=object)
temp2 = df2['访问量'][0:-1].values # array([7479078, 333371, 240919], dtype=object)
p = temp1/temp2*100
p = list(p)
p.insert(0,100) # 在0的位置插入100
p

[100, 4.457380976639099, 72.26753376868413, 45.937431252827714]

In [107]:
df2['单一环节转化率(%)'] = p
df2

,用户行为,访问量,单一环节转化率(%)
0,点击,7479078,100.000000
1,加购物车,333371,4.457381
2,收藏,240919,72.267534
3,支付,110672,45.937431


In [108]:
df2['整体转化率(%)'] = df2['访问量']/df.iloc[0,1]*100  # 其中df.iloc[0,1]为第0行第1列的数据即7479078
df2

,用户行为,访问量,单一环节转化率(%),整体转化率(%)
0,点击,7479078,100.000000,117911
1,加购物车,333371,4.457381,5255.73
2,收藏,240919,72.267534,3798.19
3,支付,110672,45.937431,1744.79


In [109]:
df2['每一环节流失率率(%)'] =100 - df2['单一环节转化率(%)']
df2

,用户行为,访问量,单一环节转化率(%),整体转化率(%),每一环节流失率率(%)
0,点击,7479078,100.000000,117911,0.000000
1,加购物车,333371,4.457381,5255.73,95.542619
2,收藏,240919,72.267534,3798.19,27.732466
3,支付,110672,45.937431,1744.79,54.062569


In [110]:
# 建立漏斗图
from pyecharts import options as opts
from pyecharts.charts import Funnel
funnel = Funnel().add(
         series_name='整体转化率(%)', 
         data_pair=[ list(z) for z in zip(df2['用户行为'],df2['整体转化率(%)']) ],
         is_selected = True,
         label_opts =opts.LabelOpts(position ='inside')
         )
funnel.set_series_opts(tooltip_opts= opts.TooltipOpts(formatter= '{a}<br/>{b}:{c}%'))
funnel.set_global_opts(title_opts= opts.TitleOpts(title='整体转化率(%)'))
# 显示图形
funnel.render_notebook()

* 通过漏斗图可以看出：  
* 在点击到收藏环节流失量最大，这是符合知觉的，因为这一阶段客户会：（1）目的不太强烈的浏览商品，看见比较中意的会加入收藏；（2）目的很强烈的搜索商品，但会陷入不同价格、款式等等的纠结中；这一环节如果加大推送量的话会增加点击的次数，同时也会增加其他行为的次数。  
    * 建议： 在客户浏览的时候及时发送优惠券，可以加大加购物车的频率。  
* 收藏/加购物车到支付阶段的流失：可能的原因是客户的预算有限或者提高预算的意愿还不够强烈，所以在比较后放弃了一些商品。  
    * 建议：在这个阶段可以加入一些优惠券或者满减的活动，来增加支付率。

### 3.1.3 复购情况分析

In [111]:
# 建立用户购买次数直方图
buy_df = data.loc[data['behavior_type'] == 4]
user_buy_s = buy_df.groupby(by = 'user_id')['behavior_type'].count()
plt.hist(user_buy_s,bins = 50)
plt.show()
user_buy_s

<Figure size 432x288 with 1 Axes>

user_id
4913          6
6118          1
7528          6
7591         21
12645         8
54056         2
63348         1
79824        13
88930        23
100539       18
104155        4
109103       17
113251        5
113960       36
120873        3
134658        2
151617       28
156608        9
157097       15
189833       22
190327        7
191366        2
213655       15
217996        2
227293        8
230711       13
239485        8
247543       22
250843       27
263670        1
             ..
142060721     4
142061900     3
142064691     9
142073213     2
142081324    14
142095821    10
142120051    47
142126535    11
142128942    14
142128951    26
142138619     4
142144275     7
142151675    13
142168798    13
142181816     8
142204924     2
142216376     9
142227202    39
142244794    15
142265405    34
142306250    10
142306361     7
142337230     4
142358910     2
142368840    11
142376113     1
142412247    11
142430177     5
142450275    39
142455899     8
Name: behavior_t

In [112]:
# 计算复购率
rebuy_rate = user_buy_s[user_buy_s>1].count() / user_buy_s.count() *100
rebuy_rate

91.44722034661264

### 3.2 用户价值分析（RMF模型：（1）对用户进行价值划分；（2）各类用户占比；）此处没有“M(money)”

In [113]:
buy_df.head()

,user_id,item_id,behavior_type,item_category,date,hour
143,101260672,73008997,4,4076,2014-11-25,13
146,116730636,85319721,4,10079,2014-12-17,11
152,104811265,61764614,4,675,2014-12-01,13
177,106230218,238910858,4,12090,2014-12-03,11
198,100684618,271840783,4,12220,2014-11-23,18


In [114]:
buy_df['date'].max() # 最近一次购买时间

Timestamp('2014-12-18 00:00:00')

In [115]:
list(buy_df.groupby(by = 'user_id')['date'])[3] #

(7591, 625981    2014-12-06
 626385    2014-11-25
 626392    2014-12-02
 626532    2014-12-04
 1605162   2014-12-10
 1605542   2014-12-12
 2979569   2014-12-12
 4473350   2014-12-04
 4473479   2014-12-03
 5438459   2014-12-10
 5438669   2014-12-12
 5752184   2014-12-12
 5752429   2014-12-13
 6839354   2014-12-13
 6839637   2014-12-10
 7140622   2014-12-11
 7140676   2014-12-10
 7140785   2014-12-06
 7770509   2014-12-11
 7770801   2014-12-04
 7770812   2014-12-11
 Name: date, dtype: datetime64[ns])

In [116]:
# 计算R：Recently
now_date = buy_df['date'].max() 
R =buy_df.groupby(by = 'user_id')['date'].apply(lambda x:now_date - x.max())/np.timedelta64(1,'D') # /np.timedelta64(1,'D')去除days
R.head()

user_id
4913     2.0
6118     1.0
7528     5.0
7591     5.0
12645    4.0
Name: date, dtype: float64

In [117]:
# 计算F
F = buy_df.groupby(by = 'user_id')['date'].count()
F.head()

user_id
4913      6
6118      1
7528      6
7591     21
12645     8
Name: date, dtype: int64

In [118]:
rfm = pd.DataFrame(data = [R,F],index = ['R','F']).T
rfm

,R,F
user_id,,
4913,2.0,6.0
6118,1.0,1.0
7528,5.0,6.0
7591,5.0,21.0
12645,4.0,8.0
54056,11.0,2.0
63348,7.0,1.0
79824,2.0,13.0
88930,1.0,23.0


In [119]:
recent_avg = rfm['R'].mean() # R的均值
freq_avg = rfm['F'].mean() # F的均值
# R应该是越小越好，则R小于均值返回1，否则返回0
def rec_value(x):
    if x < recent_avg:
        return '1'
    else:
        return '0'
# F应该是越大越好，则F大于均值返回1，否则返回0
def freq_value(x):
    if x > freq_avg:
        return '1'
    else:
        return '0'
rfm['R_value'] = rfm['R'].apply(rec_value)
rfm['F_value'] = rfm['F'].apply(freq_value)

In [120]:
rfm

,R,F,R_value,F_value
user_id,,,,
4913,2.0,6.0,1,0
6118,1.0,1.0,1,0
7528,5.0,6.0,1,0
7591,5.0,21.0,1,1
12645,4.0,8.0,1,0
54056,11.0,2.0,0,0
63348,7.0,1.0,0,0
79824,2.0,13.0,1,1
88930,1.0,23.0,1,1


In [121]:
rfm['rfm'] = rfm['R_value'].str.cat(rfm['F_value']) # 拼接R和F
rfm

,R,F,R_value,F_value,rfm
user_id,,,,,
4913,2.0,6.0,1,0,10
6118,1.0,1.0,1,0,10
7528,5.0,6.0,1,0,10
7591,5.0,21.0,1,1,11
12645,4.0,8.0,1,0,10
54056,11.0,2.0,0,0,00
63348,7.0,1.0,0,0,00
79824,2.0,13.0,1,1,11
88930,1.0,23.0,1,1,11


In [122]:
# 根据R和F的拼接来判定用户等级
def rfm_value(x):
    if x == '10': # 最近购买时间较近，购买频率低
        return '重要发展用户'
    elif x == '01': # 最近购买时间较远，购买频率高
        return '重要保持用户'
    elif x == '00':
        return '重要挽留用户'
    else:
        return '重要价值用户'
rfm['user_type'] = rfm['rfm'].apply(rfm_value)
rfm

,R,F,R_value,F_value,rfm,user_type
user_id,,,,,,
4913,2.0,6.0,1,0,10,重要发展用户
6118,1.0,1.0,1,0,10,重要发展用户
7528,5.0,6.0,1,0,10,重要发展用户
7591,5.0,21.0,1,1,11,重要价值用户
12645,4.0,8.0,1,0,10,重要发展用户
54056,11.0,2.0,0,0,00,重要挽留用户
63348,7.0,1.0,0,0,00,重要挽留用户
79824,2.0,13.0,1,1,11,重要价值用户
88930,1.0,23.0,1,1,11,重要价值用户


In [123]:
# 各类用户占比
user_type_count = rfm['user_type'].value_counts()
user_type_count / user_type_count.sum()*100

重要挽留用户    34.233626
重要发展用户    33.400855
重要价值用户    27.143822
重要保持用户     5.221697
Name: user_type, dtype: float64

In [124]:
plt.figure(figsize=(10, 8))  # 创建画布,用于置放子图
percentage_user_type = user_type_count / user_type_count.sum()*100
percentage_user_type.plot.pie(autopct='%1.1f%%', startangle=90)
plt.title('用户类型占比')

plt.show()

<Figure size 720x576 with 1 Axes>

* 结论  
* 重要发展客户：消费频次低，可以适当给点折扣或捆绑销售来增加用户的购买频率， 尽可能提高留存率。  
* 重要保持客户：消费时间间隔较远，但是消费频次高。 该类用户可能一次性购买很多东西。对于这类客户， 需要主动联系，关注他们的购物习性做精准化营销，及时满足这类用户的需求。  
* 重要价值客户：为重点用户，但用户比较少。 可以针对性地给这类客户提供VIP服务；  
* 重要挽留客户：占比最大，该类用户消费时间间隔较远， 并且消费频次低。需要主动联系客户， 调查清楚哪里出现了问题，可以通过短信、邮箱、推送等手段唤醒用户，尽可能减少损失。

### 3.3 对比分析法

In [125]:
data.head()

,user_id,item_id,behavior_type,item_category,date,hour
0,98047837,232431562,1,4245,2014-12-06,2
1,97726136,383583590,1,5894,2014-12-09,20
2,98607707,64749712,1,2883,2014-12-18,11
3,98662432,320593836,1,6562,2014-12-06,10
4,98145908,290208520,1,13926,2014-12-16,21


In [126]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8164040 entries, 0 to 8164039
Data columns (total 6 columns):
user_id          int64
item_id          object
behavior_type    int64
item_category    object
date             datetime64[ns]
hour             int64
dtypes: datetime64[ns](1), int64(3), object(2)
memory usage: 373.7+ MB


In [129]:
# 查看不同时间各种行为的分布
behavior_counts = data.groupby(['hour', 'behavior_type']).size().reset_index(name='counts')
behavior_counts.head(10)

,hour,behavior_type,counts
0,0,1,235979
1,0,2,10957
2,0,3,13652
3,0,4,4405
4,1,1,123931
5,1,2,6231
6,1,3,6537
7,1,4,1491
8,2,1,68462
9,2,2,3279


In [131]:
#查看各种行为在不同时间段的发生频率
behavior_counts_per_hour = data.groupby(['behavior_type', 'hour']).size().reset_index(name='counts')
behavior_counts_per_hour

,behavior_type,hour,counts
0,1,0,235979
1,1,1,123931
2,1,2,68462
3,1,3,46794
4,1,4,37445
5,1,5,41025
6,1,6,73035
7,1,7,183944
8,1,8,251026
9,1,9,303106


In [132]:
# 画出“点击”行为在不同时间的发生频率的折线图
behavior_type_1 = behavior_counts_per_hour[behavior_counts_per_hour['behavior_type'] == 1]

plt.figure(figsize=(10, 6))
plt.plot(behavior_type_1['hour'], behavior_type_1['counts'], marker='o')
plt.title('“点击”行为在不同时间的发生频率的折线图')
plt.xlabel('Hour')
plt.ylabel('Counts')
plt.grid(True)
plt.show()


<Figure size 720x432 with 1 Axes>

In [133]:
# 画出“收藏”行为在不同时间的发生频率的折线图
behavior_type_2 = behavior_counts_per_hour[behavior_counts_per_hour['behavior_type'] == 2]

plt.figure(figsize=(10, 6))
plt.plot(behavior_type_2['hour'], behavior_type_2['counts'], marker='o')
plt.title('“收藏”行为在不同时间的发生频率的折线图')
plt.xlabel('Hour')
plt.ylabel('Counts')
plt.grid(True)
plt.show()

<Figure size 720x432 with 1 Axes>

In [134]:
# 画出“加购物车”行为在不同时间的发生频率的折线图
behavior_type_3 = behavior_counts_per_hour[behavior_counts_per_hour['behavior_type'] == 3]

plt.figure(figsize=(10, 6))
plt.plot(behavior_type_3['hour'], behavior_type_3['counts'], marker='o')
plt.title('“加购物车”行为在不同时间的发生频率的折线图')
plt.xlabel('Hour')
plt.ylabel('Counts')
plt.grid(True)
plt.show()

<Figure size 720x432 with 1 Axes>

* 建议：点击、收藏、加购物车行为主要发生在下午17点以后，并不断上升直至到达22点左右，这其中的原因主要是17点以及后面几个小时用户下班或者放学回来以后浏览淘宝的频率变多了，这个时段可以根据对用户洗好的分析多多推送和投放相关的商品和广告，这三个行为之间有一定的连续性，如果投放及时，可以通过增加点击和收藏的频率来增加加购物车的频率。

In [135]:
# 支付”行为在不同时间的发生频率的折线图
behavior_type_4 = behavior_counts_per_hour[behavior_counts_per_hour['behavior_type'] == 4]

plt.figure(figsize=(10, 6))
plt.plot(behavior_type_4['hour'], behavior_type_4['counts'], marker='o')
plt.title('“支付”行为在不同时间的发生频率的折线图')
plt.xlabel('Hour')
plt.ylabel('Counts')
plt.grid(True)
plt.show()

<Figure size 720x432 with 1 Axes>

* 建议：可以看出，支付行为主要发生在白天，并且在中午和晚上休息时间达到高峰，建议在中午和晚上加大优惠投放力度，可以通过设置优惠有效期的方式来增加晚上支付行为发生的频率。